# Breadth-first-search на Spark RDD

https://docs.google.com/document/d/10WszM2SAWbn-2yCvefp44G7ZPDFe81RStHlMtVO793Y/edit#

### Шаг №1
Создайте SparkContext

In [1]:
import os
import sys

SPARK_HOME = "/usr/hdp/current/spark2-client"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME

PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.7-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

In [2]:
import random
from pyspark import SparkContext, SparkConf

spark_ui_port = random.choice(range(10000, 11000))
print(f"Spark UI port: {spark_ui_port}")

conf = SparkConf()
conf.set("spark.ui.port", spark_ui_port)

sc = SparkContext(appName="BFS", conf=conf)

Spark UI port: 10953


KeyboardInterrupt: 

### Шаг №2
1. Прочитайте граф из файла `/datasets/twitter/twitter_sample_small.tsv`
2. Создайте RDD, в которой граф будет представлен парами вершин
3. Убедитесь, что граф совпадает с рисунком на доске

In [ ]:
raw_graph = sc.textFile('/datasets/twitter/twitter.tsv')
# raw_graph.collect()
raw_graph = raw_graph.map(lambda x: x.split('\t'))
raw_graph = raw_graph.map(lambda x: [x[1],x[0]])

tmp = raw_graph.collect()
tmp

In [ ]:
type(raw_graph)

In [ ]:
start_point = '12'
end_point = '34'

На каждом шаге у нас есть список вершин (на шаге 0 в этот список кладём вершину start_point), рядом с которым хранится путь, по которому мы пришли в эту вершину.

Нужно:
<ol>
    <li> Для каждой вершины собрать список её соседей
    <li> Обновить пути
    <li> Проверить, не достигли ли мы искомой вершины end_point
    <li> Останов, если достигли максимальной глубины пути
</ol>
Протестируем без спарка

In [ ]:
tmp = []
with open('/home/users/datasets/twitter/twitter_sample_small.tsv','r') as f:
    for line in f:
        s = line.split('\t')
        tmp.append([s[1],s[0]])

In [ ]:
len(tmp)

In [ ]:
%%time
point_to_analyse = [['12',[]]]
shortest_paths = []
i = 0
found_flag = False
while (i < 10) & (not found_flag):
    print(i)
    res = []
    for vertex in point_to_analyse:
        for el in tmp:
            if el[0] == vertex[0]:
                tmp_path = vertex[1].copy()
                tmp_path.append(el[1][0])
                res.append([el[1][0], tmp_path])
                if el[1][0] == end_point:
                    shortest_paths.append([start_point] + tmp_path)
                    found_flag = True
    point_to_analyse = res
    i+=1
shortest_paths

# То же на спарке

Соберём всё воедино

In [ ]:
%%time
points_to_analyse = [start_point]
shortest_paths = []
found_flag = False

paths = raw_graph.filter(lambda x: x[0] in points_to_analyse).map(lambda x: [x[1],[x[0]] + [x[1]]])
i = 0
while i < 10:
    print(i)
    points_to_analyse_new = paths.map(lambda x: x[0]).collect()
    #print(points_to_analyse_new)

    if end_point in points_to_analyse_new:
        print('Eurica!')
        break

    new_vertices = raw_graph.filter(lambda x: x[0] in points_to_analyse_new)
    paths = paths.join(new_vertices).map(lambda x: [x[1][1],x[1][0] + [x[1][1]]])
#     print(new_vertices.collect())
#     print(paths.collect())

    i+=1
res = paths.filter(lambda x: x[0] == end_point).collect()
for el in res:
    print(','.join(el[1]))

In [ ]:
with open('result.csv','w') as f:
    for el in res:
        f.write(','.join(el[1]) + '\n')

In [ ]:
os.system('rm -f result.csv')

In [ ]:
'12,422,53,52,107,20,23,274,34' == ','.join(res[0][1])

In [ ]:
raw_graph.cache()
raw_graph.collect()

In [ ]:
raw_graph.filter(lambda x: x[0] in points_to_analyse).collect() #reduceByKey(lambda x,y: x+y)

In [ ]:
paths.filter(lambda x: x[0] == end_point).collect()

In [ ]:
points_to_analyse = [start_point]
shortest_paths = []
found_flag = False
paths = []

In [ ]:
paths.collect()

In [ ]:
points_to_analyse_new = paths.map(lambda x: x[0]).collect()
points_to_analyse_new

In [ ]:
new_vertices = raw_graph.filter(lambda x: x[0] in points_to_analyse_new)
new_vertices.collect()

In [ ]:
new_vertices.map(lambda x)collect()

In [ ]:
paths = paths.join(new_vertices).map(lambda x: [x[1][1],x[1][0] + [x[1][1]]])
paths.collect()

In [ ]:
if end_point in points_to_analyse_new:
    paths.filter(lambda x: x[0] == end_point).collect()

In [ ]:
points_to_analyse = points_to_analyse_new

In [ ]:
points_to_analyse, points_to_analyse_new

In [ ]:
type(paths)

In [ ]:
points_to_analyse

In [ ]:
num_vertices = vertices.count()

In [ ]:
pagerank = vertices.map(lambda x: (x, 1 / num_vertices))
pagerank.collect()

### Шаг №4
Создайте RDD, которая берет RDD с вершинами, объединяет ее с RDD с pagerank. В результате должна получится PairRDD, где ключ - это уникальная вершина, а значение - это все вершины, на которые она ссылаются и ее текущий pagerank

In [ ]:
links = graph.groupByKey().mapValues(list).cache()

In [ ]:
contributions = links.join(pagerank)
contributions.collect()

### Шаг №5
Реализуйте функцию, которая рассчитывает pagerank для всех вершин, на которые ссылается данная вершина. Функция должна быть итератором, который возвращает вершину и ее pagerank

In [ ]:
def pagerank_elements(neighbours, pagerank):
    n = len(neighbours)
    for i in neighbours:
        yield (i, pagerank / n)

### Шаг №6
Обновите RDD с pagerank значениями, посчитанными с помощью функции из предыдущего шага

In [ ]:
pagerank = contributions.flatMap(lambda x: pagerank_elements(x[1][0],x[1][1])).reduceByKey(lambda x, y: x + y)
pagerank.collect()

### Шаг №7
Напишите цикл, который проводит несколько итераций вычисления pagerank и на каждой печатает номер итерации и текущие pagerank

In [ ]:
ITERATIONS = 5

for i in range (ITERATIONS):
    links = graph.groupByKey().mapValues(list).cache()
    contributions = links.join(pagerank)
    pagerank = contributions.flatMap(lambda x: pagerank_elements(x[1][0],x[1][1])).reduceByKey(lambda x, y: x + y)
    print(f"Iter {i} of {ITERATIONS}. Current pagerank: {sorted(pagerank.collect(), key=lambda x: x[0])} ")
pagerank.collect()

In [ ]:


for i in range (0,5):
    contributions = links.join(pagerank)
    pagerank = contributions.flatMap(lambda x: pagerank_elements(x[1][0],x[1][1])).reduceByKey(lambda x, y: x + y)
    
pagerank.collect()

### Шаг №8
Не забудьте остановить SparkContext

In [ ]:
sc.stop()